# MLDS Predict Financial Strength Of Countries

## Features of the dataset:

**Country Name:** Name of the country

**Country Code:** Code assigned of the country

**Year**

**Balance:** International balance with the world bank for the country. *(To be predicted)*

**Inflation:** Inflation in percentage for that particular year

**GDP:** GDP growth for that particular year

**Exports:** Exports of the country for that particular year

**Trade:** Trade figures for the country for that particular year

In [56]:
version

               _                           
platform       x86_64-w64-mingw32          
arch           x86_64                      
os             mingw32                     
system         x86_64, mingw32             
status                                     
major          3                           
minor          5.3                         
year           2019                        
month          03                          
day            11                          
svn rev        76217                       
language       R                           
version.string R version 3.5.3 (2019-03-11)
nickname       Great Truth                 

In [57]:
# Install necessary packages

install.packages('ggplot2')
install.packages('dplyr')


Installing package into 'C:/Users/ksp/Documents/R/win-library/3.5'
(as 'lib' is unspecified)


package 'ggplot2' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\ksp\AppData\Local\Temp\RtmpGAfBz1\downloaded_packages


Installing package into 'C:/Users/ksp/Documents/R/win-library/3.5'
(as 'lib' is unspecified)


package 'dplyr' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'dplyr'"


The downloaded binary packages are in
	C:\Users\ksp\AppData\Local\Temp\RtmpGAfBz1\downloaded_packages


In [58]:
# Read datasets

train <- read.csv('https://www.dropbox.com/s/ghnw3fbf2pfmfxn/final_train_data.csv?raw=1', header = T)

test <- read.csv('https://www.dropbox.com/s/e8my78li4fs9kd6/final_test_data.csv?raw=1', header = T)

In [59]:
head(train)

id,Country.Name,Country.Code,Year,Balance,Inflation,GDP,Exports,Trade
0,Senegal,SEN,1983,-304382939,9.274213,-5.326393,618000000,59.22430
1,Togo,TGO,2009,-176669246,1.855275,3.510297,903026000,76.24254
2,Pakistan,PAK,1992,-1876092233,10.057085,7.705898,7351000000,34.48944
3,Nepal,NPL,1988,-271450571,11.815310,7.696809,190000000,24.97842
5,Malta,MLT,2016,726913236,1.604548,5.516617,3017298710,83.78147
6,Israel,ISR,2006,6883600000,1.992213,5.210821,46789400000,63.08043


In [60]:
# Examine structure of the dataset

str(train)

'data.frame':	4506 obs. of  9 variables:
 $ id          : int  0 1 2 3 5 6 7 9 10 11 ...
 $ Country.Name: Factor w/ 150 levels "Albania","Algeria",..: 116 136 103 96 87 64 96 5 18 135 ...
 $ Country.Code: Factor w/ 150 levels "ABW","AGO","ALB",..: 120 132 108 106 93 65 106 4 20 133 ...
 $ Year        : int  1983 2009 1992 1988 2016 2006 1990 2008 2013 2011 ...
 $ Balance     : num  -3.04e+08 -1.77e+08 -1.88e+09 -2.71e+08 7.27e+08 ...
 $ Inflation   : num  9.27 1.86 10.06 11.82 1.6 ...
 $ GDP         : num  -5.33 3.51 7.71 7.7 5.52 ...
 $ Exports     : num  6.18e+08 9.03e+08 7.35e+09 1.90e+08 3.02e+09 ...
 $ Trade       : num  59.2 76.2 34.5 25 83.8 ...


In [61]:
summary(train)

       id                   Country.Name   Country.Code       Year     
 Min.   :   0   Israel            :  47   ISR    :  47   Min.   :1960  
 1st Qu.:1430   South Africa      :  45   ZAF    :  45   1st Qu.:1986  
 Median :2834   Canada            :  43   CAN    :  43   Median :1997  
 Mean   :2844   Dominican Republic:  39   DOM    :  39   Mean   :1996  
 3rd Qu.:4255   Malaysia          :  39   MYS    :  39   3rd Qu.:2007  
 Max.   :5703   Saudi Arabia      :  39   SAU    :  39   Max.   :2016  
                (Other)           :4254   (Other):4254                 
    Balance             Inflation              GDP             Exports         
 Min.   :-8.060e+11   Min.   :  -29.691   Min.   :-62.076   Min.   :2.000e+06  
 1st Qu.:-9.812e+08   1st Qu.:    2.400   1st Qu.:  1.481   1st Qu.:4.804e+08  
 Median :-1.603e+08   Median :    6.184   Median :  3.800   Median :2.930e+09  
 Mean   :-5.132e+08   Mean   :   31.613   Mean   :  3.641   Mean   :4.340e+10  
 3rd Qu.: 2.885e+07   3r

Looks like, there are missing values in train. Let's count them.

In [62]:
# Check for missing values in train

na_count <- sapply(train, function(y) sum(length(which(is.na(y)))))
na_count <- data.frame(na_count)
na_count

,na_count
id,0
Country.Name,0
Country.Code,0
Year,0
Balance,0
Inflation,245
GDP,244
Exports,16
Trade,202


**Inflation, GDP, Exports & Trade columns have missing values**

In [63]:
# Find rows with Balance 0

train[train$Balance==0, c(2,4,5,6,7,8,9)]

,Country.Name,Year,Balance,Inflation,GDP,Exports,Trade
67,Maldives,1977,0,NA,NA,3390000,NA
2387,Israel,1962,0,8.901592,10.156117,279200000,36.42629
2519,Lesotho,1979,0,2.428386,2.893919,45083100,128.42485
3329,Israel,1964,0,5.433481,7.937192,372400000,35.58242


In [64]:
# First few missing rows

head(train[rowSums(is.na(train))>0,])

,id,Country.Name,Country.Code,Year,Balance,Inflation,GDP,Exports,Trade
42,56,Libya,LBY,1977,2159087462,NA,NA,11410900000,NA
46,62,Libya,LBY,1992,1407193323,NA,NA,9942000000,44.58790
50,66,Maldives,MDV,1988,9042358,NA,NA,40000000,77.10078
67,91,Maldives,MDV,1977,0,NA,NA,3390000,NA
76,102,Maldives,MDV,1984,-16300000,NA,NA,18000000,64.83808
79,105,Poland,POL,1978,-2545000000,NA,NA,12238000000,NA


In [65]:
# Percentage of missing value rows in train

round(nrow(train[rowSums(is.na(train))>0,])/nrow(train)*100,2)

[1] 6.41

In [66]:
# Subset numerical values in train data
train_num <- subset(train, select = c(1,4,5,6,7,8,9))

# Compute missing value% by column
Pmiss <- function(x) {round(sum(is.na(x))/length(x)*100,2)}

# Missing value % by column and transpose it
t(apply(train_num, 2, Pmiss))

id,Year,Balance,Inflation,GDP,Exports,Trade
0,0,0,5.44,5.42,0.36,4.48


- Checked original dataset on GDP from this link https://data.worldbank.org/indicator/NY.GDP.MKTP.CD
- These values were not recorded for those years that were missing in our dataset
- As values were not recorded, it doesn't make sense imputing them. So, replacing NA with 0

In [67]:
train_noNA <- na.omit(train)

# Check missing values now
na_count <- sapply(train_noNA, function(y) sum(length(which(is.na(y)))))
na_count <- data.frame(na_count)
na_count

,na_count
id,0
Country.Name,0
Country.Code,0
Year,0
Balance,0
Inflation,0
GDP,0
Exports,0
Trade,0


**Considered data from the year 1990 onwards**

In [68]:
min(train_noNA$Year)
max(train_noNA$Year)

[1] 1961

[1] 2016

In [69]:
train_from1990 <- train_noNA[train_noNA$Year>=1990,2:9]

nrow(train_from1990)

[1] 2971

In [70]:
# Separate datasets by Year

mycols = c('Country.Name', 'Country.Code', 'Balance', 'Inflation', 'GDP', 'Exports', 'Trade')

train_1990 <- train_from1990[train_from1990$Year==1990, mycols]
train_1991 <- train_from1990[train_from1990$Year==1991, mycols]
train_1992 <- train_from1990[train_from1990$Year==1992, mycols]
train_1993 <- train_from1990[train_from1990$Year==1993, mycols]
train_1994 <- train_from1990[train_from1990$Year==1994, mycols]
train_1995 <- train_from1990[train_from1990$Year==1995, mycols]
train_1996 <- train_from1990[train_from1990$Year==1996, mycols]
train_1997 <- train_from1990[train_from1990$Year==1997, mycols]
train_1998 <- train_from1990[train_from1990$Year==1998, mycols]
train_1999 <- train_from1990[train_from1990$Year==1999, mycols]
train_2000 <- train_from1990[train_from1990$Year==2000, mycols]
train_2001 <- train_from1990[train_from1990$Year==2001, mycols]
train_2002 <- train_from1990[train_from1990$Year==2002, mycols]
train_2003 <- train_from1990[train_from1990$Year==2003, mycols]
train_2004 <- train_from1990[train_from1990$Year==2004, mycols]
train_2005 <- train_from1990[train_from1990$Year==2005, mycols]
train_2006 <- train_from1990[train_from1990$Year==2006, mycols]
train_2007 <- train_from1990[train_from1990$Year==2007, mycols]
train_2008 <- train_from1990[train_from1990$Year==2008, mycols]
train_2009 <- train_from1990[train_from1990$Year==2009, mycols]
train_2010 <- train_from1990[train_from1990$Year==2010, mycols]
train_2011 <- train_from1990[train_from1990$Year==2011, mycols]
train_2012 <- train_from1990[train_from1990$Year==2012, mycols]
train_2013 <- train_from1990[train_from1990$Year==2013, mycols]
train_2014 <- train_from1990[train_from1990$Year==2014, mycols]
train_2015 <- train_from1990[train_from1990$Year==2015, mycols]
train_2016 <- train_from1990[train_from1990$Year==2016, mycols]


In [71]:
# Rename columns
colnames(train_1990) <- c('CountryName','CountryCode','Balance_1990','Inflation_1990','GDP_1990','Exports_1990','Trade_1990')
colnames(train_1991) <- c('CountryName','CountryCode','Balance_1991','Inflation_1991','GDP_1991','Exports_1991','Trade_1991')
colnames(train_1992) <- c('CountryName','CountryCode','Balance_1992','Inflation_1992','GDP_1992','Exports_1992','Trade_1992')
colnames(train_1993) <- c('CountryName','CountryCode','Balance_1993','Inflation_1993','GDP_1993','Exports_1993','Trade_1993')
colnames(train_1994) <- c('CountryName','CountryCode','Balance_1994','Inflation_1994','GDP_1994','Exports_1994','Trade_1994')
colnames(train_1995) <- c('CountryName','CountryCode','Balance_1995','Inflation_1995','GDP_1995','Exports_1995','Trade_1995')
colnames(train_1996) <- c('CountryName','CountryCode','Balance_1996','Inflation_1996','GDP_1996','Exports_1996','Trade_1996')
colnames(train_1997) <- c('CountryName','CountryCode','Balance_1997','Inflation_1997','GDP_1997','Exports_1997','Trade_1997')
colnames(train_1998) <- c('CountryName','CountryCode','Balance_1998','Inflation_1998','GDP_1998','Exports_1998','Trade_1998')
colnames(train_1999) <- c('CountryName','CountryCode','Balance_1999','Inflation_1999','GDP_1999','Exports_1999','Trade_1999')
colnames(train_2000) <- c('CountryName','CountryCode','Balance_2000','Inflation_2000','GDP_2000','Exports_2000','Trade_2000')
colnames(train_2001) <- c('CountryName','CountryCode','Balance_2001','Inflation_2001','GDP_2001','Exports_2001','Trade_2001')
colnames(train_2002) <- c('CountryName','CountryCode','Balance_2002','Inflation_2002','GDP_2002','Exports_2002','Trade_2002')
colnames(train_2003) <- c('CountryName','CountryCode','Balance_2003','Inflation_2003','GDP_2003','Exports_2003','Trade_2003')
colnames(train_2004) <- c('CountryName','CountryCode','Balance_2004','Inflation_2004','GDP_2004','Exports_2004','Trade_2004')
colnames(train_2005) <- c('CountryName','CountryCode','Balance_2005','Inflation_2005','GDP_2005','Exports_2005','Trade_2005')
colnames(train_2006) <- c('CountryName','CountryCode','Balance_2006','Inflation_2006','GDP_2006','Exports_2006','Trade_2006')
colnames(train_2007) <- c('CountryName','CountryCode','Balance_2007','Inflation_2007','GDP_2007','Exports_2007','Trade_2007')
colnames(train_2008) <- c('CountryName','CountryCode','Balance_2008','Inflation_2008','GDP_2008','Exports_2008','Trade_2008')
colnames(train_2009) <- c('CountryName','CountryCode','Balance_2009','Inflation_2009','GDP_2009','Exports_2009','Trade_2009')
colnames(train_2010) <- c('CountryName','CountryCode','Balance_2010','Inflation_2010','GDP_2010','Exports_2010','Trade_2010')
colnames(train_2011) <- c('CountryName','CountryCode','Balance_2011','Inflation_2011','GDP_2011','Exports_2011','Trade_2011')
colnames(train_2012) <- c('CountryName','CountryCode','Balance_2012','Inflation_2012','GDP_2012','Exports_2012','Trade_2012')
colnames(train_2013) <- c('CountryName','CountryCode','Balance_2013','Inflation_2013','GDP_2013','Exports_2013','Trade_2013')
colnames(train_2014) <- c('CountryName','CountryCode','Balance_2014','Inflation_2014','GDP_2014','Exports_2014','Trade_2014')
colnames(train_2015) <- c('CountryName','CountryCode','Balance_2015','Inflation_2015','GDP_2015','Exports_2015','Trade_2015')
colnames(train_2016) <- c('CountryName','CountryCode','Balance_2016','Inflation_2016','GDP_2016','Exports_2016','Trade_2016')


In [72]:
nrow(train_2016)
head(train_2016)

[1] 102

,CountryName,CountryCode,Balance_2016,Inflation_2016,GDP_2016,Exports_2016,Trade_2016
5,Malta,MLT,726913236,1.60454816,5.5166172,3.017299e+09,83.78147
48,Germany,DEU,287000000000,1.32921070,1.9436254,1.340000e+12,68.81783
88,Japan,JPN,187000000000,0.27355774,0.9381939,6.450000e+11,25.30679
116,Paraguay,PRY,414532813,5.27924056,4.0195550,8.493659e+09,66.53359
141,Haiti,HTI,-72459348,12.70684553,1.4399078,9.948500e+08,55.06667
151,Denmark,DNK,22459550689,-0.04644416,1.9640771,9.534593e+10,58.93944


In [73]:
# Unique Countries
length(unique(train_from1990$Country.Code))

[1] 150

In [74]:
CompleteTrain <- unique(train_from1990[,c('Country.Name','Country.Code')])
nrow(CompleteTrain)
head(CompleteTrain)

[1] 150

,Country.Name,Country.Code
2,Togo,TGO
3,Pakistan,PAK
5,Malta,MLT
6,Israel,ISR
7,Nepal,NPL
8,Argentina,ARG


In [75]:
typeof(CompleteTrain)
class(CompleteTrain)

[1] "list"

[1] "data.frame"

In [76]:
colnames(CompleteTrain) <- c('CountryName', 'CountryCode')
head(CompleteTrain)

,CountryName,CountryCode
2,Togo,TGO
3,Pakistan,PAK
5,Malta,MLT
6,Israel,ISR
7,Nepal,NPL
8,Argentina,ARG


In [77]:
# Merge datasets

final <- Reduce(function(x,y) merge(x, y, by = c('CountryCode', 'CountryName'), all.x=TRUE), 
               list(CompleteTrain, train_1990, train_1991, train_1992, train_1993, train_1994, train_1995, train_1996,
               train_1997, train_1998, train_1999, train_2000, train_2001, train_2002, train_2003,
               train_2004, train_2005, train_2006, train_2007, train_2008, train_2009, train_2010,
               train_2011, train_2012, train_2013, train_2014, train_2015, train_2016))


In [78]:
class(final)

[1] "data.frame"

In [79]:
str(final)

'data.frame':	150 obs. of  137 variables:
 $ CountryCode   : Factor w/ 150 levels "ABW","AGO","ALB",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ CountryName   : Factor w/ 150 levels "Albania","Algeria",..: 7 3 1 5 6 4 8 9 21 16 ...
 $ Balance_1990  : num  NA -2.36e+08 -1.18e+08 4.55e+09 NA ...
 $ Inflation_1990: num  NA 0.301 -0.469 2076.793 NA ...
 $ GDP_1990      : num  NA -0.3 -9.58 -2.4 NA ...
 $ Exports_1990  : num  NA 3.91e+09 2.30e+08 1.24e+10 NA ...
 $ Trade_1990    : num  NA 54.7 29 11.6 NA ...
 $ Balance_1991  : num  NA -5.80e+08 NA -6.47e+08 NA ...
 $ Inflation_1991: num  NA 170 NA 133 NA ...
 $ GDP_1991      : num  NA -1.2 NA 12.7 NA ...
 $ Exports_1991  : num  NA 3.41e+09 NA 1.20e+10 NA ...
 $ Trade_1991    : num  NA 39.3 NA 10.7 NA ...
 $ Balance_1992  : num  NA -7.35e+08 NA -5.55e+09 NA ...
 $ Inflation_1992: num  NA 249.1 NA 11.9 NA ...
 $ GDP_1992      : num  NA -6.9 NA 11.9 NA ...
 $ Exports_1992  : num  NA 3.76e+09 NA 1.22e+10 NA ...
 $ Trade_1992    : num  NA 101 NA 11.8 NA ...


In [80]:
head(final)

CountryCode,CountryName,Balance_1990,Inflation_1990,GDP_1990,Exports_1990,Trade_1990,Balance_1991,Inflation_1991,GDP_1991,...,Balance_2015,Inflation_2015,GDP_2015,Exports_2015,Trade_2015,Balance_2016,Inflation_2016,GDP_2016,Exports_2016,Trade_2016
ABW,Aruba,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
AGO,Angola,-235500000,0.3009027,-0.300000,3.9100e+09,54.73365,-579600000,169.905533,-1.200000,...,-10272841903,-4.0176477,3.006981,33181130226,52.32371,-3070971745,27.42131,-0.6653533,27306200000,41.79405
ALB,Albania,-118300000,-0.4687074,-9.575640,2.3000e+08,29.02516,NA,NA,NA,...,-978125860,0.1000312,2.226581,1929552771,55.13723,NA,NA,NA,NA,NA
ARG,Argentina,4552000000,2076.7934730,-2.398959,1.2353e+10,11.62273,-647000000,132.953228,12.669710,...,-17621985168,24.5453565,2.647271,56788000000,19.93200,-14693041563,40.67531,-2.2453441,57732800000,20.77853
ARM,Armenia,NA,NA,NA,NA,NA,NA,NA,NA,...,-272427725,1.2139099,3.200000,1485300000,44.76783,NA,NA,NA,NA,NA
ATG,Antigua and Barbuda,-30985185,1.6502636,3.011646,2.1000e+07,60.06933,-30922222,2.606167,2.176873,...,92810679,2.4587683,4.060219,66261989,38.58929,NA,NA,NA,NA,NA


In [81]:
# NAs are introduced in cells after merging the datasets. Let's count the NAs

final_NAcount <- sapply(final, function(y) sum(length(which(is.na(y)))))
final_NAcount <- data.frame(final_NAcount)
head(final_NAcount)

,final_NAcount
CountryCode,0
CountryName,0
Balance_1990,57
Inflation_1990,57
GDP_1990,57
Exports_1990,57


In [82]:
length(final[is.na(final)])

[1] 5395

In [84]:
final[is.na(final)] <- 0

length(final[is.na(final)])

[1] 0

In [85]:
str(final)

'data.frame':	150 obs. of  137 variables:
 $ CountryCode   : Factor w/ 150 levels "ABW","AGO","ALB",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ CountryName   : Factor w/ 150 levels "Albania","Algeria",..: 7 3 1 5 6 4 8 9 21 16 ...
 $ Balance_1990  : num  0.00 -2.36e+08 -1.18e+08 4.55e+09 0.00 ...
 $ Inflation_1990: num  0 0.301 -0.469 2076.793 0 ...
 $ GDP_1990      : num  0 -0.3 -9.58 -2.4 0 ...
 $ Exports_1990  : num  0.00 3.91e+09 2.30e+08 1.24e+10 0.00 ...
 $ Trade_1990    : num  0 54.7 29 11.6 0 ...
 $ Balance_1991  : num  0.00 -5.80e+08 0.00 -6.47e+08 0.00 ...
 $ Inflation_1991: num  0 170 0 133 0 ...
 $ GDP_1991      : num  0 -1.2 0 12.7 0 ...
 $ Exports_1991  : num  0.00 3.41e+09 0.00 1.20e+10 0.00 ...
 $ Trade_1991    : num  0 39.3 0 10.7 0 ...
 $ Balance_1992  : num  0.00 -7.35e+08 0.00 -5.55e+09 0.00 ...
 $ Inflation_1992: num  0 249.1 0 11.9 0 ...
 $ GDP_1992      : num  0 -6.9 0 11.9 0 ...
 $ Exports_1992  : num  0.00 3.76e+09 0.00 1.22e+10 0.00 ...
 $ Trade_1992    : num  0 101 0 11.

## Lets model data with linear regression